In [13]:
import psycopg2
import pandas as pd
import random
import re


In [82]:
def get_paraphrase(word,cursor):
    paraphrase = cursor.execute("SELECT paraphrase FROM public.ppdb WHERE LOWER(phrase) ILIKE LOWER(%s) AND entailment ILIKE %s", (' ' + word + ' ', ' '+ 'OtherRelated'))
    results = cursor.fetchall()
    return results

def getBool():
    boolList = [True, False, True, False]
    randBool = random.choice(boolList)
    return randBool


def getIndexes(sentence):
    wordList = sentence.split()
    indexes_without_at = [index for index, word in enumerate(wordList) if not word.startswith('@')]
    return wordList,indexes_without_at

def paraphraseWords(wordList,indexes_without_at,cursor):
    for i in indexes_without_at:
        word = wordList[i]
        paraphrase = get_paraphrase(word,cursor)
        if paraphrase and getBool():
            rand = random.choice(paraphrase)
            wordList[i] = rand[0].strip()
    return wordList

def paraphrase(sentence,cursor):
    wordList,indexes_without_at = getIndexes(sentence)
    wordList = paraphraseWords(wordList,indexes_without_at,cursor)
    sentence = ' '.join(wordList)
    return sentence

In [84]:
conn = psycopg2.connect(
   database="ppdb",
    user='postgres',
    password='password',
    host='localhost',
    port= '5432'
)
cursor = conn.cursor()
test = get_paraphrase(word="Hello",cursor=cursor)
print(test)
removableWords = [' me',' the',' a',' of']

[(' morning ',), (' greetings ',), (' yes ',), (' bonjour ',), (' greeting ',), (' goodbye ',)]


In [ ]:
query = "Show me the patients with a diagnosis of @diagnosis"
test = paraphrase(query,cursor)
print(test)

shown me the subjects with a diagnostic of @diagnosis


In [86]:
# reads from QuestionsToExpand.csv and appends to QuestionsExpanded.csv
df = pd.read_csv('QuestionsToExpand.csv', sep=',', quotechar='\'')
# gets unique paraphrases for each question until 2 most recent paraphrases are not unique
for i in range(len(df)):
    question = df.iloc[i]['Question']
    paraphrases = []
    nonunique = 0
    while len(paraphrases) < 20:
        print(len(paraphrases))
        paraphrased = paraphrase(question,cursor)
        if paraphrased not in paraphrases:
            paraphrases.append(paraphrased)
        else:
            index = len(paraphrases)
            if nonunique == index:
                break
            nonunique = index
    df2 = pd.DataFrame({'Question': paraphrases,'SQL': df.iloc[i]['SQL']})
    df2.to_csv('QuestionsExpanded.csv', mode='a', header=False)
    paraphrases = []
    


0


1
2
3
4
5
6
7
7
8
9
9
10
11
12
13
13
14
14
15
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
